# External Lab 

Here each question is of 1 mark.

# Multiple Linear Regression

## Problem Statement

Use Multiple Linear Regression to **predict the consumption of petrol** given relevant variables are the petrol tax, the per capita, income, the number of miles of paved highway, and the proportion of the population with driver's licenses.

## Dataset

There are 48 rows of data.  The data include:

      I,  the index;
      A1, the petrol tax;
      A2, the per capita income;
      A3, the number of miles of paved highway;
      A4, the proportion of drivers;
      B,  the consumption of petrol.

### Reference 

    Helmut Spaeth,
    Mathematical Algorithms for Linear Regression,
    Academic Press, 1991,
    ISBN 0-12-656460-4.

    S Weisberg,
    Applied Linear Regression,
    New York, 1980, pages 32-33.

## Question 1 - Exploratory Data Analysis

*Read the dataset given in file named **'petrol.csv'**. Check the statistical details of the dataset.*

**Hint:** You can use **df.describe()**

In [7]:
### Import functions to read file
import numpy as np
import pandas as pd
fueldata = pd.read_csv("petrol.csv")
fueldata.columns = fueldata.columns.str.strip()
## Show first 2 values
fueldata.head(2)

,tax,income,highway,dl,consumption
0,9.0,3571,1976,0.525,541
1,9.0,4092,1250,0.572,524


In [8]:
fueldata.describe()

,tax,income,highway,dl,consumption
count,48.000000,48.000000,48.000000,48.000000,48.000000
mean,7.668333,4241.833333,5565.416667,0.570333,576.770833
std,0.950770,573.623768,3491.507166,0.055470,111.885816
min,5.000000,3063.000000,431.000000,0.451000,344.000000
25%,7.000000,3739.000000,3110.250000,0.529750,509.500000
50%,7.500000,4298.000000,4735.500000,0.564500,568.500000
75%,8.125000,4578.750000,7156.000000,0.595250,632.750000
max,10.000000,5342.000000,17782.000000,0.724000,968.000000


# Question 2 - Cap outliers 

Find the outliers and cap them. (Use (Q1 - 1.5 * IQR) as the minimum cap and (Q3 + 1.5 * IQR) as the max cap. The decision criteria is you should consider the datapoints which only falls within this range. The data points which fall outside this range are outliers and the entire row needs to be removed

In [9]:
### Use (Q1 - 1.5 * IQR) as the min cap and (Q3 + 1.5 * IQR) as the max cap to identify outlier 

# Reading the file and removing spaces from column names
fueldata = pd.read_csv("petrol.csv")
fueldata.columns = fueldata.columns.str.strip()

# variables to count number of rows and columns
df_nrows = len(fueldata.index)
df_ncols = len(fueldata.columns)

# creating flag columns for each variable with Null Values in it
fueldata['tax_f']=np.nan
fueldata['income_f']=np.nan
fueldata['highway_f']=np.nan
fueldata['dl_f']=np.nan
fueldata['consumption_f']=np.nan

# Outer loop to set the IQR, Max range and Min Range to define the outlier
for i in range(df_ncols):
    dfcol = fueldata.iloc[:,i]
    IQR_c = (dfcol.quantile(.75)) - (dfcol.quantile(.25))
    max_c = ((dfcol.quantile(.75)) + (1.5 * IQR_c))
    min_c = ((dfcol.quantile(.25)) - (1.5 * IQR_c))
    
    # for each column validation, run the loop on the row to flag
    for j in range(df_nrows):
        if (fueldata.iloc[j,i] < min_c) or (fueldata.iloc[j,i] > max_c):
            fueldata.iloc[j,i+5] = 'N'
        else:
            fueldata.iloc[j,i+5] = 'Y'

# Set a consolidated flag to identify a row with even one outlier from possible values
fueldata['flag'] = fueldata['tax_f']+fueldata['income_f']+fueldata['highway_f']+fueldata['dl_f']+fueldata['consumption_f']
# Create subset dataframe for any further calculations in this assignment
fueldata_e = fueldata[fueldata['flag']=='YYYYY'][['tax','income','highway','dl','consumption']]

In [10]:
fueldata_e.head(2)

,tax,income,highway,dl,consumption
0,9.0,3571,1976,0.525,541
1,9.0,4092,1250,0.572,524


# Question 3 - Independent variables and collinearity 
Which attributes seems to have stronger association with the dependent variable consumption?

In [13]:
# Run Correlation function on all columns to identify the dependencies
fueldata_e.corr()

,tax,income,highway,dl,consumption
tax,1.000000,-0.109537,-0.390602,-0.314702,-0.446116
income,-0.109537,1.000000,0.051169,0.150689,-0.347326
highway,-0.390602,0.051169,1.000000,-0.016193,0.034309
dl,-0.314702,0.150689,-0.016193,1.000000,0.611788
consumption,-0.446116,-0.347326,0.034309,0.611788,1.000000


### Observing the above correlation values between all the variables, we can see that there is stronger association between the number of drivers and consumption. And comparatively tax has an association in a negative way. 
Insights :
As tax increases the consumption decreases.
As number of drivers is more consumption is more

# Question 4 - Transform the dataset 
Divide the data into feature(X) and target(Y) sets.

In [14]:
x = fueldata[['tax','dl']]

In [15]:
y = fueldata[['consumption']]

# Question 5 - Split data into train, test sets 
Divide the data into training and test sets with 80-20 split using scikit-learn. Print the shapes of training and test feature sets.

In [16]:
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=7)

In [17]:
x_train.shape

(38, 2)

In [18]:
y_train.shape

(38, 1)

In [19]:
x_test.shape

(10, 2)

In [20]:
y_test.shape

(10, 1)

# Question 6 - Build Model 
Estimate the coefficients for each input feature. Construct and display a dataframe with coefficients and X.columns as columns

In [21]:
linreg=LinearRegression()
linreg.fit(x_train,y_train)
coef_df = pd.DataFrame(columns = x.columns,
                      data = linreg.coef_)
coef_df

,tax,dl
0,-35.408416,1338.598782


In [38]:
predict_y = linreg.predict(x_test)
mse = metrics.mean_squared_error(predict_y, y_test)
Rmse = np.sqrt(mse)
print(Rmse)

52.922138692778375


# R-Square 

# Question 7 - Evaluate the model 
Calculate the accuracy score for the above model.

In [22]:
# Accuracy of the model on the training set
linreg.score(x_train,y_train)

0.5598159154951787

In [39]:
# Accuracy of the model on the test set
linreg.score(x_test,y_test)

0.4305607542599543

# Question 8: Repeat the same Multi linear regression modelling by adding both Income and Highway features
Find R2 


In [23]:
x = fueldata[['tax','income','highway','dl']]
y = fueldata[['consumption']]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=7)

In [24]:
x_train.shape

(38, 4)

In [25]:
y_train.shape

(38, 1)

In [26]:
x_test.shape

(10, 4)

In [27]:
y_test.shape

(10, 1)

# Question 9: Print the coefficients of the multilinear regression model

In [28]:
linreg.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [29]:
linreg.coef_

array([[-3.86347479e+01, -6.86557766e-02, -2.86388372e-03,
         1.40238439e+03]])

# Question 10 
In one or two sentences give reasoning on R-Square on the basis of above findings
Answer

### *R squared value increase if we increase the number of independent variables to our analysis

In [97]:
linreg.score(x_train,y_train)

0.6937373436237997

In [40]:
linreg.score(x_test,y_test)

0.4305607542599543